## 3. 转换器架构
这一章主要介绍转换器架构。虽然整体来说比较难理解，但它是生成式AI取得突破的根本原因，所以将其放在较靠前的章节。但从应用的角度来说，不理解转换器架构也没有太大关系。所以建议读者可以先粗读一遍，不管理解不理解都可以直接进入后续章节的学习。
学习转换器架构的关键是理解**多头自注意力（Multi-Head Self-Attention）**机制，其本质是以更为精准的向量来表示文本的语义，从而提升生成式AI的处理效率。

### 3.1 文本向量化
以向量的形式表示文本是AI能够理解人类语言的基础，但文本的向量化并不是简单地将文本中的每个字符映射到一个数值来组成向量，而是先将文本拆分为单词，再对单词进行向量化，最后将单词向量进行拼接，得到最终的文本向量。

#### 3.1.1 分词
AI中的单词或词语并不一定与自然语言中的单词或词语完全一致，它们本质上是出现频率较高的字符组合。所以为了与自然语言中的单词或词语相区分，在AI中一般将它们称为词元（Tokens）。绝大多数情况下，词元与单词或词语一致，但有些词元可能与单词或词语不同。比如生成式AI的英文“generative AI”中，generative就不是一个词元，而genera却是一个词元。因为在generative，generation，generator，general等等单词中都有genera这个字符组合，所以它的出现频率就更高一些，会被机器学习算法识别为词元。
这一小节主要介绍了三种常用的分词算法，即BPE（Byte-Pair Encoding）、WordPiece、SentencePiece。它们具体实现虽然不难但也不是特别容易理解，读者可以大致了解它们的基本思想即可。

#### 3.1.2 分词器
transformers库使用分词器（Tokenizer）负责分词，可通过AutoTokenizer加载预训练模型的分词器。本小节通过加载不同模型的分词器，向读者展示了不同分词算法处理词元的结果。
1. BERT模型采用了WordPiece算法
2. GPT-2模型采用了BPE算法
3. T5使用的分词算法为SentencePiece算法
使用下面的代码并替换模型名称，即可查看不同模型分词器的处理结果：

In [ ]:
from transformers import AutoTokenizer
# 选择预训练模型的名称
model_name = "bert-base-uncased"
# 加载预训练的分词器
tokenizer = AutoTokenizer.from_pretrained(model_name)
# 输出分词器的类型
print(tokenizer.tokenize("I love generative AI"))
# 获取BERT词汇表
voc = tokenizer.get_vocab()
# 查看genera和generate在词汇表中的数值ID
print(voc["genera"], voc["generate"])

#### 3.1.3  独热向量与嵌入向量
词元的向量形式有两种，一种是独热向量，另一种是嵌入向量。独热（One Hot）向量使用一个与词汇表长度相同的向量表示词元，整个向量中仅词元对应位置的分量为1，其它分量则全部为0。独热向量可以体现词元语义的独立性，但也存在着明显的问题。其一，独热向量是典型的稀疏向量，对于大模型数以万计的词汇表来说是巨大的空间浪费。另一个问题就是独热向量两两正交，无法体现词元之间在语义上的相似性。词嵌入（Word Embedding）是对独热向量的一种改进，它将词元映射到一个低维的语义空间中，每个维度可以看成是词元某一语义特征的强度，相似词元可以在相关维度上有相近似的强度，从而体现词元之间在某一语义特征上的相似性。所以词嵌入向量可以更准确地反映词元的语义相似性，是更适合AI处理的向量化文本输入。
本节举了一个非常有名的词元向量运算公式，可以让我们对词嵌入有一个直观的感知，那就是“king - man + woman ≈ queen”。理解了这个例子，对于文本向量化的理解就会更加清晰。

### 3.2 词嵌入算法简介*